# Tensor shapes in Pyro

这一部分主要介绍pyro是如何组织tensor的dimensions的。

## Distribution shapes

对于一个分布，其有两个shapes：即batch_shape和event_shape。

In [1]:
import torch
import pyro.distributions as pdist
import torch.distributions as tdist

In [2]:
pgaussion = pdist.Normal(0, 1)
tgaussion = tdist.Normal(0, 1)

In [3]:
# 注意，隔离pyro.distributions和torch.distributions行为是一致的
print("batch_shape:", pgaussion.batch_shape, tgaussion.batch_shape)
print("event_shape:", pgaussion.event_shape, tgaussion.event_shape)

batch_shape: torch.Size([]) torch.Size([])
event_shape: torch.Size([]) torch.Size([])


In [4]:
x = pgaussion.sample()
assert x.shape == pgaussion.batch_shape + pgaussion.event_shape

In [5]:
# 注意，torch.Size对象的`__add__`方法是concat到一起而不是加在一起，其类似list
torch.Size([1, 2]) + torch.Size([3])

torch.Size([1, 2, 3])

实际上，我们还有另外一个shape：sample_shape，其在使用`sample`方法的时候出现：

In [6]:
x2 = pgaussion.sample([3])
assert x2.shape == torch.Size([3]) + pgaussion.batch_shape + pgaussion.event_shape

所以我们得到的样本的shape和以上3个shape间的关系是：
```
      |      iid     | independent | dependent
------+--------------+-------------+------------
shape = sample_shape + batch_shape + event_shape
```

下面我们来具体介绍这3个shapes：

- `sample_shape`表示的是样本的维度，比如`sample_shape=[3, 4]`表示我们一共随机了产生了12个独立的样本，然后把每一个样本排列成$3\times4$的矩阵形式。
- `batch_shape`表示的是参数的维度（这里说是independent的维度，也是合理的），我们可以设置参数是`torch.tensor([2, 3])`，则`batch_shape=[2]`，也就是说会产生2个分布，这两个分布是相互独立的，其中一个分布的参数是2，另一个分布的参数是3。
- `event_shape`表示的是多维分布的那个"多维"，比如协方差不为0的多维正态分布等等，这些维度上的各个值间不是独立的，因为会收到其他值的影响。

注意：

1. `log_prob`函数只会考虑到sample_shape和batch_shape，不管多少维的随机变量，其密度函数只会映射到$[0, 1]$区间中。
2. 一般来说，`torch.Size([])`和`torch.Size([1])`是不一样的，对于`Normal`，其`event_shape==torch.Size([])`，表示其作为一个单维度的随机变量没有`event_shape`，而对于`MultivariateNormal`，虽然我们也可以让其只有一维，但`event_shape==torch.Size([1])`，表示其是一个多维随机变量，什么时候都有`event_shape`。

下面是一些例子：

In [7]:
# torch.Size()对象`__equal__`方法可以对tuple进行操作
d = pdist.Bernoulli(0.5)
assert d.batch_shape == ()
assert d.event_shape == ()
x = d.sample()
assert x.shape == ()
assert d.log_prob(x).shape == ()

In [8]:
d = pdist.Bernoulli(0.5 * torch.ones(3, 4))
assert d.batch_shape == (3, 4)
assert d.event_shape == ()
x = d.sample()
assert x.shape == (3, 4)
assert d.log_prob(x).shape == (3, 4)

In [9]:
# 还可以通过`expand()`方法来形成batch_shape，此方法的效果是在当前batch_shape的基础上进一步扩增batch_shape
d = pdist.Bernoulli(torch.tensor([0.1, 0.2, 0.3, 0.4])).expand([3, 4])
assert d.batch_shape == (3, 4)
assert d.event_shape == ()
x = d.sample()
assert x.shape == (3, 4)
assert d.log_prob(x).shape == (3, 4)

In [10]:
# Multivariate distribution有nonempty的`.event_shape`
d = pdist.MultivariateNormal(torch.zeros(3), torch.eye(3, 3))
assert d.batch_shape == ()
assert d.event_shape == (3,)
x = d.sample()
assert x.shape == (3,)
assert d.log_prob(x).shape == ()

In [11]:
# if there is nonempty sample_size，then...
x2 = d.sample([4])
assert x2.shape == (4, 3)
assert d.log_prob(x2).shape == (4,)

## Reshaping distributions

在pyro中我们可以有方法`.to_event(n)`来把某个`batch_shape`转变成`event_shape`，也就是把一些独立的分布看做是一个联合分布。这在torch中没有。

In [12]:
d = pdist.Bernoulli(0.5 * torch.ones(3, 4)).to_event(1)
assert d.batch_shape == (3,)
assert d.event_shape == (4,)
x = d.sample()
assert x.shape == (3, 4)
assert d.log_prob(x).shape == (3,)

In [13]:
# 这个转换必须从右边开始，里面的参数不是第几个，而是一共要转换几个维度
d = pdist.Bernoulli(0.5 * torch.ones(3, 4)).to_event(2)
assert d.batch_shape == ()
assert d.event_shape == (3, 4)
x = d.sample()
assert x.shape == (3, 4)
assert d.log_prob(x).shape == ()

将多个独立的分布看成一个整个的联合分布，这个操作在pyro中会非常频繁的使用。其有两个优势：

1. 这使得我们可以简单的构建一个多维分布。
2. 这可以使我们避免去使用`plate`来声明一个Multivariate distribution的独立性。

（当然，这并不一定是个好处，因为被`plate`声明了独立性后，在进行梯度估计的时候可以受益于这个独立性，而使用`to_event()`方式构建的多维独立分布在pyro的眼中并不是独立的，pyro将其作为dependent variables来看待。）

In [14]:
# 第一个版本
import pyro

x = pyro.sample("x", pdist.Normal(0, 1).expand([10]).to_event(1))
assert x.shape == (10,)

In [15]:
# 第二个版本
with pyro.plate("x_plate", 10):
    x = pyro.sample("x", pdist.Normal(0, 1))
    assert x.shape == (10,)

注意，将independent variables看做是dependent，永远都是safe的（特别是对于那些可以reparametric的distributions，此trick使得即使增加了dependence也不会降低效率）。而将dependent variables看做是independent则会导致一些问题。

## 使用`plate`来declaring independence

我们可以使用`plate`的上下文管理器来指定特定的batch dimensions是independence。这些independence可能会被inference algorithm利用从而降低Enforce estimation的variance。

注意，虽然plate可以操作某个batch dimension是independent，但其并没有将该dimension放入到event_shape中。真正将dimension放入event_shape中的操作是`to_event`，但其并没有指定independent（尽管其实就是independent）。那我们之所以需要使用`plate`来进行操作，只是为了能够利用到其inference时的trick而已。

有几种利用`plate`的方式：

第一，最简单的，直接在`sample`上使用，则其最右边的batch dimension被设定为independent

In [16]:
with pyro.plate("my_plate"):
    ss = pyro.sample("x", pdist.Normal(torch.arange(12).reshape(3, 4).float(), 1.))
    print(ss)

tensor([[-0.8772,  1.6277,  2.5118,  3.3601],
        [ 4.4994,  3.4269,  6.5692,  5.7114],
        [ 7.6142,  8.5268, 10.8181, 12.1643]])


第二，我们提供最右边的batch dimension在`plate`中作为一个参数，其效果和上面的一致

In [17]:
with pyro.plate("my_late", 4):
    ss = pyro.sample("x", pdist.Normal(torch.arange(12).reshape(3, 4).float(), 1.))
    print(ss)

tensor([[-0.1892,  1.5685,  1.1956,  3.7006],
        [ 4.3691,  6.6751,  6.7053,  6.1233],
        [ 7.5525,  8.4695, 10.7730, 10.4904]])


In [18]:
with pyro.plate("my_late", 2):
    ss = pyro.sample("x", pdist.Normal(torch.tensor([1.0, 2.0]), 1.))
    print(ss)

tensor([-1.2492,  0.5607])


In [19]:
# 这种方法的优势1：如果只有一个batch dimension，而且就想让这个batch dimension是independent，而且这个dimension就是多个distribution的
#   简单重复，则可以不需要在下面的`sample`语句中指定这个dimension
with pyro.plate("my_late", 2):
    ss = pyro.sample("x", pdist.Normal(1.0, 1.0))
    print(ss)

tensor([0.0652, 1.9002])


In [20]:
# 这种方法的优势2：可以通过嵌套的`plate`上下文管理器，来实现多个维度的independent
with pyro.plate("x_axis", 3):  # 注意，这是第-1个dimension
    with pyro.plate("y_axis", 2):  # 注意，这是第-2个dimension
        ss = pyro.sample("x", pdist.Normal(torch.arange(6).reshape(1, 2, 3).float(), 1.))
        print(ss)

tensor([[[-0.0460,  1.4014,  1.0916],
         [ 3.4536,  3.6544,  3.6335]]])


注意到，之前的教程中，将多个观测的独立性赋予到模型中时使用的也是`plate`，其出现了这种用法（当时介绍了2种，其中的第二种--矢量运算）。

In [36]:
data = torch.tensor([1, 0, 1])
with pyro.plate("my_late", 3):
    ss1 = pyro.sample("x1", pdist.Bernoulli(0.5), obs=data)
    ss2 = pyro.sample("x2", pdist.Bernoulli(0.5).expand([3]), obs=data)
print(ss1, ss2)

tensor([1, 0, 1]) tensor([1, 0, 1])


第三，使用`plate`中的参数`dim`来指定哪一个维度是independent，这使得我们可以通过合理的组织来完成复杂的依赖关系。

In [37]:
def model1():
    a = pyro.sample("a", pdist.Normal(0, 1))
    b = pyro.sample("b", pdist.Normal(torch.zeros(2), 1).to_event(1))
    with pyro.plate("c_plate", 2):
        c = pyro.sample("c", pdist.Normal(torch.zeros(2), 1))
    with pyro.plate("d_plate", 3):
        d = pyro.sample("d", pdist.Normal(torch.zeros(3, 4, 5), 1).to_event(2))
    assert a.shape == ()           # batch_shape == (), event_shape == (), pyro independ == (), real independ == ()
    assert b.shape == (2,)         # batch_shape == (), event_shape == (2,), pyro independ == (), real independ == (2,)
    assert c.shape == (2,)         # batch_shape == (2,), event_shape == (), pyro independ == (2,), real independ == (2,)
    assert d.shape == (3, 4, 5)    # batch_shape == (3, ), event_shape == (4, 5), pyro independ == (4, 5), real independ == (3, 4, 5)

    x_axis = pyro.plate("x_axis", 3, dim=-2)
    y_axis = pyro.plate("y_axis", 2, dim=-3)
    with x_axis:  # 注意不能再嵌套一层list等等
        x = pyro.sample("x", pdist.Normal(0, 1))
    with y_axis:
        y = pyro.sample("y", pdist.Normal(0, 1))
    with x_axis, y_axis:
        xy = pyro.sample("xy", pdist.Normal(0, 1))
        z = pyro.sample("z", pdist.Normal(0, 1).expand([5]).to_event(1))
    assert x.shape == (3, 1)         # batch_shape == (3, 1), event_shape == (), pyro independ == (3,), real independ == (3, 1)
    assert y.shape == (2, 1, 1)      # batch_shape == (2, 1, 1), event_shape == (), pyro independ == (2,), real independ == (2, 1, 1)
    assert xy.shape == (2, 3, 1)     # batch_shape == (2, 3, 1), event_shape == (), pyro independ == (2, 3), real independ == (2, 3, 1)
    assert z.shape == (2, 3, 1, 5)   # batch_shape == (2, 3, 1), event_shape == (5,), pyro independ == (2, 3), real independ == (2, 3, 1, 5)

In [38]:
def test_model(model, guide, loss):
    pyro.clear_param_store()
    loss.loss(model, guide)

In [39]:
from pyro.infer import Trace_ELBO

test_model(model1, model1, Trace_ELBO())

注意到，在使用`log_prob`的时候，event_shape会被sum out到一起，剩下的batch_shape（和sample_shape）会保留下来。

In [40]:
import pyro.poutine as poutine

trace = poutine.trace(model1).get_trace()
trace.compute_log_prob()
print(trace.format_shapes())

Trace Shapes:            
 Param Sites:            
Sample Sites:            
       a dist       |    
        value       |    
     log_prob       |    
       b dist       | 2  
        value       | 2  
     log_prob       |    
 c_plate dist       |    
        value     2 |    
     log_prob       |    
       c dist     2 |    
        value     2 |    
     log_prob     2 |    
 d_plate dist       |    
        value     3 |    
     log_prob       |    
       d dist     3 | 4 5
        value     3 | 4 5
     log_prob     3 |    
  x_axis dist       |    
        value     3 |    
     log_prob       |    
  y_axis dist       |    
        value     2 |    
     log_prob       |    
       x dist   3 1 |    
        value   3 1 |    
     log_prob   3 1 |    
       y dist 2 1 1 |    
        value 2 1 1 |    
     log_prob 2 1 1 |    
      xy dist 2 3 1 |    
        value 2 3 1 |    
     log_prob 2 3 1 |    
       z dist 2 3 1 | 5  
        value 2 3 1 | 5  
     log_pro

In [41]:
d = pdist.Normal(torch.ones(3, 4), 1.0).to_event(1)
x = d.sample()
d.log_prob(x)

tensor([-9.3470, -4.0367, -5.3413])

## Subsampling tensors inside a `plate`

之前说到过，`plate`的一个作用是用来做subsampling的。实际上做subsampling是需要一些条件的，比如必须是独立的，不然expected loss of subsamples并不会成比例的减小。所以一般来说subsampling只会用在样本那一层级。

但现在因为数据结构的原因，我们使用`plate`将一些维度也规定成independent，则subsampling也是可以使用的。（尽管理论上是这样，但实际上这个subsampling也只会应用到data那里）

使用时，我们需要即指定数据的总数量，而且还要指定`subsample_size`：

In [42]:
data = torch.arange(100.)
def model2():
    mean = pyro.param("mean", torch.zeros(len(data)))
    with pyro.plate("data", len(data), subsample_size=10) as ind:  # subsampling的逻辑需要在整个模型中体现出来
        assert len(ind) == 10
        batch = data[ind]
        mean_batch = mean[ind]
        x = pyro.sample("x", pdist.Normal(mean_batch, 1), obs=batch)
        assert len(x) == 10

test_model(model2, guide=lambda: None, loss=Trace_ELBO())

## Broadcasting to allow parallel enumeration

这里，pyro可以对一些discrete latent variables进行并行运算，这可以显著的降低gradient estimators’ variance。这里进行一下解释：

1. 如果我们的model中存在2个finite discrete variables，第一个的类别数是2，第二个的类别是3，则这两个的联合分布可以看做是一个6分类的variables。
2. 通过上面的讨论，不管有几个finite discrete variables，我们都可以将其看做是一个finite discrete variables，则我们在进行estimate的时候，可以先估计一个分类上的loss或gradient，这大大减小了其方差，然后在通过一些方法将多个类别上的估计组合在一起。
3. 所以我们必须提供一些方法能够在model中存在finite discrete variables的时候，能够对它们进行枚举，把这些类别都枚举出来，然后分别计算、estimate。
4. 那么，我们就需要独立出一些维度来储存这些枚举，比如还是上面2个finite discrete variables的例子，我们需要设计两个维度来储存这两个变量的枚举（无论这两个变量是否有关系）。
5. 为了方便，这些储存枚举的维度会被放在所有维度的前面，但有时候我们还需要一些维度作为batch dimensions来**指定independent、用于表达不同的层次或者作为data的那个维度**，所有我们需要下面的`max_plate_nesting`，其相当于提供了一个batch dimensions的预算。

而为了能够进行这个parallel enumeration，pyro需要知道哪些tensor dimensions可以被enumerate，所以我们需要指定一个`max_plate_nesting`。这个值作为一个参数被送入`TraceEnum_ELBO()`中。

指定此参数为2后，则能够使用`plate`来指定独立性的batch dimensions是从right往left数的2个维度了，如果再多那一个维度来指定独立性则会出错。剩下的batch dimensions会被看做是enumeration dimension。

比如`model1`中，如果我们指定`max_plate_nesting=2`，则会保存，因为xy和z的-2、-3维度被指定为independent（使用`with plate("y", 2, dim=-3)`）。而如果我们指定`max_plate_nesting=3`或`4`则是安全的。

另外我们还需要使用`pyro.infer.config_enumerate`装饰器来进行装饰model。

In [75]:
from pyro.infer import config_enumerate, TraceEnum_ELBO

@config_enumerate
def exam():
    p = pyro.param("p", torch.arange(6.) / 6)                      # 足球比赛中有6类运动员，比如前锋、后卫等等。每类运动员有自己在本场比赛中会进球的几率，即p
    a = pyro.sample("a", pdist.Categorical(torch.ones(6) / 6))     # 这六类运动员在比赛上是等量分布的，所以这里相当于随机选择一个运动员，看其是哪一类
    with pyro.plate("plate1", 3):                                  # 然后我们随机选择了3个人（我们空闲出batch dimension的预算，就是为了放这样的维度的）
        b = pyro.sample("b", pdist.Bernoulli(p[a]))                # 这是根据市每个人进球的概率再进行采样，得到3个人是否进球的一个sample
    print(a.shape, b.shape)
exam()

torch.Size([]) torch.Size([3])


In [76]:
test_model(exam, lambda: None, TraceEnum_ELBO(max_plate_nesting=2))

torch.Size([6, 1, 1]) torch.Size([2, 1, 1, 1])


通过上面的实验，我们看到：

1. 第一个出现的discrete variable只有一个枚举维度，而第二个出现的discrete variable就需要2个枚举维度，因为需要把上一个也包括进去。
2. 单纯运行model，其不会出现枚举维度，但在使用了`TraceEnum_ELBO`并指定了`max_plate_nesting`后，才会出现枚举维度（也就是说计算loss的时候）。
3. 第二个discrete variable尽管有2个枚举维度，但其第一个枚举维度（这其实是为第一个discrete variable准备的）的shape=1，只有第二个枚举维度的shape是其类别数。
4. 对于所有的离散变量，其`plate`不再起作用，详细情况可以见下面的c、d（两者虽然都在`plate`下面，但其batch shape size都是1，而对于连续性变量e，则不会受到枚举的影响，该是怎么样还是怎么样，下面e之所以有那么多的维度，是收到其接受的参数e的影响），这是为了节约储存资源，但这也意味着如果我们想要进行进一步的处理，必须额外进行操作，把discrete variables的batch dimensions填上。

In [89]:
@config_enumerate
def model3():
    p = pyro.param("p", torch.arange(6.) / 6)
    locs = pyro.param("locs", torch.tensor([-1., 1.]))

    a = pyro.sample("a", pdist.Categorical(torch.ones(6) / 6))                     # 
    b = pyro.sample("b", pdist.Bernoulli(p[a]))
    with pyro.plate("c_plate", 4):
        c = pyro.sample("c", pdist.Bernoulli(0.3))
        with pyro.plate("d_plate", 5):
            d = pyro.sample("d", pdist.Bernoulli(0.4))
            e_loc = locs[d.long()].unsqueeze(-1)
            e_scale = torch.arange(1., 8.)
            e = pyro.sample("e", pdist.Normal(e_loc, e_scale).to_event(1))
            f = pyro.sample("f", pdist.Normal(1, 1))

    #                   enumerated/batch/event dims
    assert a.shape == (         6, 1, 1   )
    assert b.shape == (      2, 1, 1, 1   )
    assert c.shape == (   2, 1, 1, 1, 1   )
    assert d.shape == (2, 1, 1, 1, 1, 1,  )
    assert e.shape == (2, 1, 1, 1, 5, 4, 7)
    assert f.shape == (            5, 4   )

    assert e_loc.shape   == (2, 1, 1, 1, 1, 1, 1,)
    assert e_scale.shape == (                  7,)

test_model(model3, model3, TraceEnum_ELBO(max_plate_nesting=2))

可以看到，我们至多嵌套了2层`plate`。 

In [46]:
trace = poutine.trace(poutine.enum(model3, first_available_dim=-3)).get_trace()
trace.compute_log_prob()
print(trace.format_shapes())

Trace Shapes:                
 Param Sites:                
            p             6  
         locs             2  
Sample Sites:                
       a dist             |  
        value       6 1 1 |  
     log_prob       6 1 1 |  
       b dist       6 1 1 |  
        value     2 1 1 1 |  
     log_prob     2 6 1 1 |  
 c_plate dist             |  
        value           4 |  
     log_prob             |  
       c dist           4 |  
        value   2 1 1 1 1 |  
     log_prob   2 1 1 1 4 |  
 d_plate dist             |  
        value           5 |  
     log_prob             |  
       d dist         5 4 |  
        value 2 1 1 1 1 1 |  
     log_prob 2 1 1 1 5 4 |  
       e dist 2 1 1 1 5 4 | 7
        value 2 1 1 1 5 4 | 7
     log_prob 2 1 1 1 5 4 |  


In [88]:
from pyro.distributions import constraints
from pyro.distributions.unit import broadcast_shape

width = 8                                                               # 图片的宽度
height = 10                                                             # 图片的高度
sparse_pixels = torch.tensor([[3, 2], [3, 5], [3, 9], [7, 1]]).long()   # 这张图片的稀疏表示，在这些坐标上这些点是1
enumerated = None

def fun(observe):                                                       # 现在建模这张图片的生成过程
    p_x = pyro.param("p_x", torch.tensor(0.1), constraint=constraints.unit_interval)
    p_y = pyro.param("p_y", torch.tensor(0.1), constraint=constraints.unit_interval)
    x_axis = pyro.plate("x_axis", width, dim=-2)
    y_axis = pyro.plate("y_axis", height, dim=-1)
    
    with x_axis: 
        x_active = pyro.sample("x_active", pdist.Bernoulli(p_x))        # p(x=1)  这两个分布可以认为是那个hidden variable，实际上它不能看做是像素在x（或y）轴上的边际分布，所以
    with y_axis:                                                        #         其实际意义未知。但根据后面的表示，可以认为pixel=1的概率依赖于这两个hidden variables，所以可以
        y_active = pyro.sample("y_active", pdist.Bernoulli(p_y))        # p(x=1)  暂时将其看成一个未知含义的隐含变量来理解
    if enumerated:
        assert x_active.shape == (2, 1, 1)                              # 基于上面描述的enumerate的规则，这里width和height并没有表现
        assert y_active.shape == (2, 1, 1, 1)
    else:
        assert x_active.shape == (width, 1)                             # 这里显示，如果`plate`仅指定了倒数第二个维度，则也会出现倒数第一个维度，那个维度的shape是1
        assert y_active.shape == (height,)
        
    # broadcast
    p = 0.1 + 0.5 * x_active * y_active                                 # p(pixel=1|x=1,y=1)，这一步有广播操作
    if enumerated:
        assert p.shape == (2, 2, 1, 1)
    else:
        assert p.shape == (width, height)
    dense_pixels = p.new_zeros(broadcast_shape(p.shape, (width, height)))    # 这里是生成一个0tensor，其shape是(width, height)或(2, 2, width, height)，用来填充稀疏表示作为obs
    
    # ellipsis slicing
    for x, y in sparse_pixels:
        dense_pixels[..., x, y] = 1                                          # `...`表示前面还有n个维度，在numpy中有看到
    if enumerated:
        assert dense_pixels.shape == (2, 2, width, height)                   # 前面的2x2的维度表示的是(x, y)的4种情况，相当于将前面discrete variables的情况都保存到samples中，便于计算
    else:                                                                    # 在这四种情况下，obs都是一样的
        assert dense_pixels.shape == (width, height)
    
    with x_axis, y_axis:
        if observe:
            pyro.sample("pixels", pdist.Bernoulli(p), obs=dense_pixels)
            

def model4():
    fun(observe=True)
    
def guide4():
    fun(observe=False)
    
enumerated = False
test_model(model4, guide4, Trace_ELBO())

enumerated = True
test_model(model4, config_enumerate(guide4, "parallel"), TraceEnum_ELBO(max_plate_nesting=2))

In [92]:
# 以下的例子证明了，`plate`可以自动满足我们对于dimension的诸多设置

num_particles = 100
width = 8
height = 10
sparse_pixels = torch.tensor([[3, 2], [3, 5], [3, 9], [7, 1]]).long()

def sample_pixel_locations_no_broadcasting(p_x, p_y, x_axis, y_axis):
    with x_axis:
        x_active = pyro.sample("x_active", pdist.Bernoulli(p_x).expand([num_particles, width, 1]))
    with y_axis:
        y_active = pyro.sample("y_active", pdist.Bernoulli(p_y).expand([num_particles, 1, height]))
    return x_active, y_active

def sample_pixel_locations_full_broadcasting(p_x, p_y, x_axis, y_axis):
    with x_axis:
        x_active = pyro.sample("x_active", pdist.Bernoulli(p_x))
    with y_axis:
        y_active = pyro.sample("y_active", pdist.Bernoulli(p_y))
    return x_active, y_active
    
def sample_pixel_locations_partial_broadcasting(p_x, p_y, x_axis, y_axis):
    with x_axis:
        x_active = pyro.sample("x_active", pdist.Bernoulli(p_x).expand([width, 1]))
    with y_axis:
        y_active = pyro.sample("y_active", pdist.Bernoulli(p_x).expand([height]))
    return x_active, y_active
    
def fun(observe, sample_fn):
    p_x = pyro.param("p_x", torch.tensor(0.1), constraint=constraints.unit_interval)
    p_y = pyro.param("p_y", torch.tensor(0.1), constraint=constraints.unit_interval)
    x_axis = pyro.plate("x_axis", width, dim=-2)
    y_axis = pyro.plate("y_axis", height, dim=-1)
    
    with pyro.plate("num_particle", 100, dim=-3):
        x_active, y_active = sample_fn(p_x, p_y, x_axis, y_axis)
        assert x_active.shape == (2, 1, 1, 1)
        assert y_active.shape == (2, 1, 1, 1, 1)
        p = 0.1 + 0.5 * x_active * y_active
        assert p.shape == (2, 2, 1, 1, 1)
        
        dense_pixels = p.new_zeros(broadcast_shape(p.shape, (width, height)))
        for x, y in sparse_pixels:
            dense_pixels[..., x, y] = 1
        assert dense_pixels.shape == (2, 2, 1, width, height)
        
        with x_axis, y_axis:
            if observe:
                pyro.sample("pixels", pdist.Bernoulli(p), obs=dense_pixels)

def test_model_with_sample_fn(sample_fn):
    def model():
        fun(observe=True, sample_fn=sample_fn)
    
    @config_enumerate
    def guide():
        fun(observe=False, sample_fn=sample_fn)
        
    test_model(model, guide, TraceEnum_ELBO(max_plate_nesting=3))
    
test_model_with_sample_fn(sample_pixel_locations_no_broadcasting)
test_model_with_sample_fn(sample_pixel_locations_full_broadcasting)
test_model_with_sample_fn(sample_pixel_locations_partial_broadcasting)